<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Aula_6_Outputs_em_RDDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7 and Java 8. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Run a local spark session to test your installation:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

# Reading a CSV from google drive

Utilizando o Google Colab, é possível importar os datasets diretamente do Google Drive, sem ter que realizar o upload manual dos mesmos para a instância colab manualmente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
spark.read\
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("drive/My\ Drive/My\ Professional\ Carrer/Spark\ course/virtual_classroom/colab_test/test.csv") \
  .show()

# Outputs em RDDs
Como visto nas aulas anteriores, uma output é uma **ação** que delimita um **job** e dá origem de fato a execução das **transformações** previamente encadeadas. Essa operação pode ser realizada de duas formas:
1. Trazendo dados para o driver da aplicação
2. Salvando dados em uma fonte externa



In [ ]:
sc = spark.sparkContext

### Coleção a ser utilizada
Durante essa parte do curso, veremos como aplicar os comandos utilizando a seguinte coleção em Python (texto extraído da seção Overwiew em http://spark.apache.org/docs/latest/rdd-programming-guide.html#overview

In [ ]:
text = "At a high level, every Spark application consists of a driver program that runs the user’s main function and executes various parallel operations on a cluster. The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing Scala collection in the driver program, and transforming it. Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. Finally, RDDs automatically recover from node failures. A second abstraction in Spark is shared variables that can be used in parallel operations. By default, when Spark runs a function in parallel as a set of tasks on different nodes, it ships a copy of each variable used in the function to each task. Sometimes, a variable needs to be shared across tasks, or between tasks and the driver program. Spark supports two types of shared variables: broadcast variables, which can be used to cache a value in memory on all nodes, and accumulators, which are variables that are only added to, such as counters and sums"
words = text.split(" ")

In [ ]:
words_rdd = sc.parallelize(words)

### Outputs/ações como estruturas em Python
##### rdd.reduce()
Reduz um RDD de qualquer tipo a um único número. Por exemplo, uma soma de números

In [ ]:
array = range(1, 21)

sc.parallelize(array) \
    .reduce(lambda x, y: x + y)

210

Ou retornar a maior palavra no nosso RDD de words

In [ ]:
("spark", "hadoop", "elasticsearch")
("hadoop", "elasticsearch")
("elasticsearch")


def word_leght_reducer(left_word, right_word):
  if len(left_word) > len(right_word):
    return left_word
  else:
    return right_word

words_rdd.reduce(word_leght_reducer)

words_rdd.reduce(lambda x, y: word_leght_reducer(x, y))


'Hadoop-supported'

##### rdd.count()
Conta a quantidade de linhas. Além do `.count()`, também há `.max()` e `.min()`.

In [ ]:
words_rdd.count()

214

##### rdd.take()
Retorna uma quantidade N de linhas do RDD em uma lista em Python no driver. Variações são `.takeOrdered()`, `.takeSample()` e `.top()`.

In [ ]:
words_rdd.take(10)

##### rdd.collect()
Retorna todo o RDD como uma lista de elementos para o driver. Os mesmos cuidados ao se tormar com o `.collect()` em Dataframes são mais do que válidos aqui também

In [ ]:
#words_rdd.collect()

### Outputs/ações em Arquivos
##### rdd.saveAsTextFile()
O comando `.saveAsTextFile()` é uma ação que visa persistir o RDD em formato de um arquivo de texto simples (plain-text). Não é possível, utilizando esse método, salvar nos formatos que vimos nas aulas de Dataframes tais como JSON, CSV, Parquet. Tal feito, requer um recurso que veremos em breve

In [ ]:
words_rdd \
    .repartition(2) \
    .saveAsTextFile("./output/words_txt_2")

In [ ]:
!ls ./output/words_txt

In [ ]:
!head -n 5 ./output/words_txt/part-00000

### Output/Ação utilizando o foreach
O comando `.foreach()` basicamente tem a função de iterar sobre as linhas de um RDD. Como é uma ação, e não uma transformação, o comando `.foreach()` **não retorna nenhum valor**. É útil quando se está trabalhando com **Accumulators** ou deseja-se escrever o dado em uma fonte externa. Grande parte dos outputs em formatos de arquivos distintos ou bancos de dados utilizados na API high-level dos Dataframes utiliza essa ação como base para construir a API.

Como se usar essa ação. Basta definir uma função passá-la como parâmetro para o método foreach():
```python
def my_funct(row):
    # TODO
    ...

rdd.foreach(my_funct)
```

CUIDADO: Erros comuns ao utilizar o comando `.foreach()`:

1. Atualização de variáveis globais convencionais. Cada executor tem a sua JVM e a variável global definida no driver não estará disponível para os executores, gerando valores inconsistentes:

```python
counter = 0
rdd = sc.parallelize(data)

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x

rdd.foreach(increment_counter)
print("Counter value: ", counter)
```

2. Printando elementos usando o `foreach`. Assumindo que a execução está ocorrendo em modo cluster, cada executor pode estar em uma máquina distinta e a execução da função print gerará uma escrita em stdout que ocorrerá dentro do executor e não do driver

```python
rdd.foreach(print)
```


### Veja mais
É possível encontrar uma lista de todas as ações disponíveis para uso nos RDDs na API do Spark: http://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

In [ ]:
words_rdd.foreach(print)

In [ ]:
counter = 0
 
# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += 1
 
words_rdd.foreach(increment_counter)
print("Counter value: ", counter)

Counter value:  0
